In [1]:
import pandas as pd
from My_model import My_model
from tqdm import tqdm
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据加载

In [2]:
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence
import pickle

# 载入Seqs字典和smiles2vec模型
sentence_model = word2vec.Word2Vec.load('updated_smi2vec_model.pkl')
file_path = 'seq_word_dict.pkl'
with open(file_path, 'rb') as f:
    seq_word_dict = pickle.load(f)
    seq_word_dict = seq_word_dict

In [3]:
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
import random

# 加载datas数据
datas = torch.load('datas.pt')

# 固定随机种子以确保结果可复现
random.seed(42)

# 打乱数据
random.shuffle(datas)

# 设置训练集和测试集的比例
train_ratio = 0.9  # 90%的数据作为训练集
train_size = int(train_ratio * len(datas))

# 划分数据集
train_dataset = datas[:train_size]
test_dataset = datas[train_size:]

# 随机划分数据集
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 数据处理

In [4]:
from rdkit import Chem
from torch.nn.utils.rnn import pad_sequence
import mytools

word_num = len(seq_word_dict)
train_smi_padded_embeddings = []
train_seqs = []
for data in tqdm(train_loader, desc='训练数据处理中'):
    smi = data.smi
    mols = [Chem.MolFromSmiles(s) for s in smi]
    sentences = [mol2alt_sentence(m, 1) for m in mols]
    smi_embeddings = [torch.tensor(sentence_model.wv[s], dtype=torch.float32) for s in sentences]
    padded_embeddings = pad_sequence(smi_embeddings, batch_first=True)
    train_smi_padded_embeddings.append(padded_embeddings)
    seqs = [torch.tensor(mytools.split_sequence(s, 3, seq_word_dict), dtype=torch.long) for s in data.seq]
    train_seqs.append(seqs)

test_smi_padded_embeddings = []
test_seqs = []
for data in tqdm(test_loader, desc='测试数据处理中'):
    smi = data.smi
    mols = [Chem.MolFromSmiles(s) for s in smi]
    sentences = [mol2alt_sentence(m, 1) for m in mols]
    smi_embeddings = [torch.tensor(sentence_model.wv[s], dtype=torch.float32) for s in sentences]
    padded_embeddings = pad_sequence(smi_embeddings, batch_first=True)
    test_smi_padded_embeddings.append(padded_embeddings)
    seqs = [torch.tensor(mytools.split_sequence(s, 3, seq_word_dict), dtype=torch.long) for s in data.seq]
    test_seqs.append(seqs)

测试数据处理中: 100%|██████████| 68/68 [00:04<00:00, 16.59it/s]


# 模型导入、训练

In [9]:
import numpy as np
import csv
import torch
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt
import warnings


# 模型加载
model = My_model(9, 16, 128, 300, len(seq_word_dict), 300, 128, heads=2).to(device)

# 定义损失函数，对于回归任务通常使用均方误差损失(MSEL)
criterion = torch.nn.MSELoss()

# 选择优化器，这里使用Adam，你可以根据需要调整学习率等参数
opti = torch.optim.Adam(model.parameters(), lr=1e-4)

# 忽略特定的警告
warnings.filterwarnings("ignore")

In [10]:
# CSV文件路径
metrics_csv_path = '/data/coding/metrics.csv'

# 定义CSV的列名
fieldnames = ['Epoch', 'Train_Loss', 'Val_Loss', 'Train_RMSE', 'Val_RMSE', 'Train_R2', 'Val_R2', 'Train_MAE', 'Val_MAE']

#设置训练次数epoch
num_epochs = 100
train_losses = []  # 用于存储每轮的训练损失
test_losses = []  # 用于存储每轮的测试损失
maes_train = []  # 用于存储每轮训练集的MAE
rmses_train = []  # 用于存储每轮训练集的RMSE
r2s_train = []  # 用于存储每轮训练集的R²
maes_test = []  # 用于存储每轮测试集的MAE
rmses_test = []  # 用于存储每轮测试集的RMSE
r2s_test = []  # 用于存储每轮测试集的R²

best_r2_val = -np.inf  # 初始化最佳验证集R²为负无穷大
best_model_wts = None  # 用于存储最佳模型权重

# 初始化CSV文件
with open(metrics_csv_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

for epoch in num_epochs:
    model.train()
    model.to(device)
    running_loss_train = 0.0
    all_preds_train = []
    all_targets_train = []

    # 训练阶段
    for data, spe, seqs in zip(train_loader, train_smi_padded_embeddings, train_seqs):
        data = data.to(device)
        spe = spe.to(device)
        labels = data.kcat
        opti.zero_grad()
        out = model(data, spe, seqs)
        loss = criterion(out, labels)
        loss.backward()
        opti.step()
        running_loss_train += loss.item()
        all_preds_train.append(out.detach().cpu().numpy())
        all_targets_train.append(labels.cpu().numpy())

    epoch_loss_train = running_loss_train / len(train_loader)
    all_preds_train = np.concatenate(all_preds_train).flatten()
    all_targets_train = np.concatenate(all_targets_train).flatten()
    
    mae_train = mean_absolute_error(all_targets_train, all_preds_train)
    rmse_train = np.sqrt(mean_squared_error(all_targets_train, all_preds_train))
    r2_train = r2_score(all_targets_train, all_preds_train)

    # 测试阶段
    model.eval()
    with torch.no_grad():
        running_loss_val = 0.0
        all_preds_val = []
        all_targets_val = []

        for data, spe, seqs in zip(test_loader, test_smi_padded_embeddings, test_seqs):
            data = data.to(device)
            spe = spe.to(device)
            labels = data.kcat
            out = model(data, spe, seqs)
            val_loss = criterion(out, labels)
            running_loss_val += val_loss.item()
            all_preds_val.append(out.detach().cpu().numpy())
            all_targets_val.append(labels.cpu().numpy())

        epoch_loss_val = running_loss_val / len(test_loader)
        all_preds_val = np.concatenate(all_preds_val).flatten()
        all_targets_val = np.concatenate(all_targets_val).flatten()
        
        mae_val = mean_absolute_error(all_targets_val, all_preds_val)
        rmse_val = np.sqrt(mean_squared_error(all_preds_val, all_targets_val))
        r2_val = r2_score(all_targets_val, all_preds_val)

        # 更新最佳模型
        if r2_val > best_r2_val:
            best_r2_val = r2_val
            best_model = model.cpu().eval()  # 确保模型在CPU上保存，并处于评估模式

        # 记录本轮的指标
        train_losses.append(epoch_loss_train)
        maes_train.append(mae_train)
        rmses_train.append(rmse_train)
        r2s_train.append(r2_train)
        test_losses.append(epoch_loss_val)
        maes_test.append(mae_val)
        rmses_test.append(rmse_val)
        r2s_test.append(r2_val)

    # 写入CSV文件
    with open(metrics_csv_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({
            'Epoch': epoch + 1,
            'Train_Loss': epoch_loss_train,
            'Val_Loss': epoch_loss_val,
            'Train_RMSE': rmse_train,
            'Val_RMSE': rmse_val,
            'Train_R2': r2_train,
            'Val_R2': r2_val,
            'Train_MAE': mae_train,
            'Val_MAE': mae_val
        })

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss_train:.4f}, Val Loss: {epoch_loss_val:.4f}, '
          f'Train RMSE: {rmse_train:.4f}, Val RMSE: {rmse_val:.4f}, Train R²: {r2_train:.4f}, Val R²: {r2_val:.4f}, '
          f'Train MAE: {mae_train:.4f}, Val MAE: {mae_val:.4f}')

# # 在训练结束后保存最佳模型
# if best_model is not None:
#     # 保存整个模型到文件
#     torch.save(best_model, '/data/coding/best_reaction_graph_model.pth')
# else:
#     print("No best model found.")

  1%|          | 1/100 [00:29<48:19, 29.29s/it]

Epoch 1/100, Train Loss: 2.3049, Val Loss: 2.1953, Train RMSE: 1.5182, Val RMSE: 1.4816, Train R²: 0.0414, Val R²: 0.0710, Train MAE: 1.1752, Val MAE: 1.1360


  2%|▏         | 2/100 [00:58<47:57, 29.37s/it]

Epoch 2/100, Train Loss: 2.1631, Val Loss: 2.0701, Train RMSE: 1.4709, Val RMSE: 1.4384, Train R²: 0.1002, Val R²: 0.1243, Train MAE: 1.1324, Val MAE: 1.1204


  3%|▎         | 3/100 [01:28<47:31, 29.40s/it]

...

Epoch 98/100, Train Loss: 1.4319, Val Loss: 1.4815, Train RMSE: 1.1966, Val RMSE: 1.2172, Train R²: 0.4045, Val R²: 0.3730, Train MAE: 0.9058, Val MAE: 0.9334


 99%|█████████▉| 99/100 [48:24<00:29, 29.42s/it]

Epoch 99/100, Train Loss: 1.4377, Val Loss: 1.8897, Train RMSE: 1.1990, Val RMSE: 1.3750, Train R²: 0.4021, Val R²: 0.1999, Train MAE: 0.9126, Val MAE: 0.9426


100%|██████████| 100/100 [48:53<00:00, 29.34s/it]

Epoch 100/100, Train Loss: 1.4228, Val Loss: 1.4633, Train RMSE: 1.1928, Val RMSE: 1.2097, Train R²: 0.4083, Val R²: 0.3807, Train MAE: 0.9067, Val MAE: 0.9273
